# Stereovision

![Suzanne](main.png)

Stereovision is a discipline that deals with the reconstruction of 3D information from images. For the reconstruction of a point, several images of this point are needed. These images must be taken from different points of view. The key step of the reconstruction, which is often problematic, is to identify the images of the point to be reconstructed in each view.

## Epipolar Geometry

Epipolar geometry involves two cameras. The epipolar geometry describes the geometric properties between two views of the same scene and depends only on the intrinsic parameters of the cameras and their relative positions. It provides, in particular, the epipolar constraint, which will be very useful to produce the matches between views.

## The Fondamental Matrix

![Epipolar Geometry - Sanyam Kapoor](epipolar.png)

Let us imagine that we have two images, right and left, of the world space. Let's take a point $\vec{x}$ in the right image space. The point $\vec{X}$ of the world space, of which $\vec{x}$ is the image, can be anywhere on the line passing through $\vec{x}$ and the optical center of the right camera. We will call this line the back-projected ray of $\vec{x}$. Let us note $\vec{x}'$ the image of $\vec{X}$ in the left image space. The locus of $\vec{x}'$ is therefore the image line of the back-projected ray of $\vec{x}$. This line is called the epipolar line and is denoted $\vec{l}'$. The epipolar line passes through the epipole $\vec{e}'$, image of the optical center of the right camera.

In 2D projective geometry, a line with equation $ax+by+c = 0$ is represented by a vector with three components $(a, b, c)^T$ defined to within one factor. Thus, we have the following relationship:

>The point $\vec{x}$ belongs to the line $\vec{l}$ if and only if $x^T\vec{l} = 0$.

Moreover, in 2D projective geometry, the following remarkable relations are valid:

- The intersection of two lines $l$ and $l'$ is given by $x = l \times l'$,
- The line passing through two points $x$ and $x'$ is given by $l = x \times x'$.

Note that the vector product can be written as a product of matrix $x \times y = [x]_\times y$ where

$$[x]_\times = \begin{pmatrix} 0 & −x3 & x2 \\ x3 & 0 & −x1 \\ −x2 & x1 & 0 \end{pmatrix}$$

To find the equation of the epipolar line in the left image space, we just need to find the coordinates of two points of this line. The first is the image $P'\vec{C}$ of the optical center $\vec{C}$ of the right camera where $P'$ is the projection matrix of the left camera. The second is $P'P^{+}\vec{x}$ where $P^{+}$ is the pseudo inverse of the projection matrix $P$ of the right camera. The epipolar line thus has the equation $l' = [P'\vec{C}]_\times{}P'P^{+}\vec{x} = F\vec{x}$ with $F = [P'\vec{C}]_\times{}P'P^{+}$. $F$ is called fundamental matrix.

Since the epipolar line $\vec{l}' = F\vec{x}$ is the locus of $\vec{x}'$, $\vec{x}'$ therefore belongs to $\vec{l}'$ which leads to the epipolar constraint :

>**The fundamental matrix is such that for any pair of points corresponding $\vec{x} \leftrightarrow \vec{x}'$ in the two images, we have $\vec{x}'^{T}F\vec{x} = 0$.**

## Computation of the fundamental matrix

The fundamental matrix $F$ has seven degrees of freedom. It has nine components but these are defined to within one scale factor, which removes one degree of freedom. Moreover, the matrix $F$ is a singular matrix ($det(F) = 0$) which gives us seven degrees of freedom. So we need at least seven correspondences to compute $F$. The equation $x'^{T}_iFx_i = 0$ and the seven correspondences allow us to write a system of equations of the form $Af = 0$, where $f$ is the vector which contains the components of the matrix $F$. Let us assume that $A$ is a 7×9 matrix of rank 7. The general solution of $Af = 0$ can be written $\alpha f_1 + (1-\alpha) f_2$ where $f_1$ and $f_2$ are two particular independent solutions of $Af = 0$. We then use the singularity constraint $det(\alpha F_1 + (1 - \alpha)F_2) = 0$ to determine $\alpha$. Since the singularity constraint gives rise to a third degree equation, we may have one or three solutions for $F$.

## OpenCV

In practice you will use the OpenCV library. In python, you have access to its functions through the `cv2` module.

You can find help with the calibration and reconstruction functions on the site https://docs.opencv.org/4.0.0/d9/d0c/group__calib3d.html

## Goal

In the zip of the statement you will find two sequences of images taken by two cameras during the scanning of an object by a laser plane.

![Laser](scanRight/scan0010.png)

You will also find shots of a checkerboard in different positions that will help you calibrate your cameras.

![Damier](chessboards/c2Right.png)

The goal is to reconstruct the scanned object in 3D.

In [10]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D

#Etape 1 
#Recuper les données de nos deux caméras

#Prepa
chessboardSize = (7,7)
frameSize = (1920,1080)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.

imagesLeft = sorted(glob.glob('chessboards/c*L*.png'))
imagesRight = sorted(glob.glob('chessboards/c*R*.png'))
for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))

#rotation matrix => convert vector to matrix
rmatLeft = cv.Rodrigues(rvecsL[2])[0]
rmatRight = cv.Rodrigues(rvecsR[2])[0]

#full [R|t] matrix => add t in R
rotMatRight = np.concatenate((rmatRight,tvecsR[0]), axis=1)
rotMatLeft = np.concatenate((rmatLeft,tvecsL[0]), axis=1)

#camera matrix (A [R|t])
camLeft = cameraMatrixL @ rotMatLeft
camRight = cameraMatrixR @ rotMatRight

# find cx and cy for both cameras
camWorldCenterLeft = np.linalg.inv(np.concatenate((rotMatLeft,[[0,0,0,1]]), axis=0)) @ np.transpose([[0,0,0,1]])
camWorldCenterRight = np.linalg.inv(np.concatenate((rotMatRight,[[0,0,0,1]]), axis=0)) @ np.transpose([[0,0,0,1]])

flags = 0
flags = cv.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rotMatrix, transMatrix, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objectPoints=objpoints, 
    imagePoints1=imgpointsL, imagePoints2=imgpointsR, cameraMatrix1 = newCameraMatrixL, distCoeffs1= distL, 
    cameraMatrix2=newCameraMatrixR, distCoeffs2=distR, imageSize= grayL.shape[::-1], R=rmatLeft, T= tvecsL[0], flags=flags, criteria=criteria_stereo)


#full [R|t] matrix => add t in R
newRotMatLeft = np.concatenate((rotMatrix, transMatrix), axis=1)
newCamLeft = newCameraMatrixL @ newRotMatLeft
newCamCenterLeft = np.linalg.inv(np.concatenate((newRotMatLeft,[[0,0,0,1]]), axis=0)) @ np.transpose([[0,0,0,1]])

#Parametres intrinsèque, skew = 0
    #Assume pixel is square -> focalLengthX == focalLengthY
focalLengthX = newCameraMatrixL[0,0]
focalLengthY = newCameraMatrixR[1,1]
    #Assume prinicpal point is at the center of the image -> img not cropped
principalPointX = newCameraMatrixL[0,2]
principalPointY = newCameraMatrixR[1,2]

print(fundamentalMatrix)

[[-1.47041321e-08 -1.22159145e-06  8.36303231e-04]
 [ 3.07075828e-06  6.07967918e-08 -1.40288079e-02]
 [-1.92262090e-03  1.23214259e-02  1.00000000e+00]]


In [11]:
#Etape 2

# 1. For each pixel in first image find corresponding epipolar line in second images
# 2. Examine all corresponding pixels on epipolar line and pick best match
# 3. Triangulate matches to get depth information

#Stereo rectification : on met les deux caméra à même "hauteur" => leur epilines vont coincider == scanlines
#Step 1 : Find projective transformation HLeft and HRight (3x3) such that epipoles e and e' are mapped to the infinite point [1,0,0]^T

rectRotLeft, rectRotRight, rectProjLeft, rectProjRight, Q, roi1, roi2 = cv.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rotMatrix, transMatrix)

mapLeft1, mapLeft2 = cv.initUndistortRectifyMap(newCameraMatrixL, distL, rectRotLeft, rectProjLeft, grayL.shape[::-1], cv.CV_32FC1)
mapRight1, mapRight2 = cv.initUndistortRectifyMap(newCameraMatrixR, distR, rectRotRight, rectProjRight, grayR.shape[::-1], cv.CV_32FC1)

#Load images
imagesLeft = glob.glob('scanLeft/*.png')
imagesRight = glob.glob('scanRight/*.png')

#remap images
def remap(imagesLeft, mapLeft1, mapLeft2, imagesRight, mapRight1, mapRight2):
    remapLeft = []
    remapRight = []
    for i in range(len(imagesLeft)):
        imgL = cv.imread(imagesLeft[i])
        imgR = cv.imread(imagesRight[i])
        remapLeft.append(cv.remap(imgL, mapLeft1, mapLeft2, cv.INTER_LINEAR))
        remapRight.append(cv.remap(imgR, mapRight1, mapRight2, cv.INTER_LINEAR))
        
    return remapLeft, remapRight

remapLeft, remapRight = remap(imagesLeft, mapLeft1, mapLeft2, imagesRight, mapRight1, mapRight2)

def displayImagesRectified(remapLeft, remapRight):
    for i in range(len(remapLeft)):
        cv.resize(remapLeft[i], Size(), fx=0.5, fy=0.5)
        cv.resize(remapRight[i], (0,0), fx=0.5, fy=0.5)
        cv.imshow('rectifiedLeft', remapLeft[i])
        cv.imshow('rectifiedRight', remapRight[i])
        cv.waitKey(0)
        cv.destroyAllWindows()